# ES → BBF Migration Data Analysis

This notebook analyzes all data required for the ES to BBF Salesforce migration.

## Driving Principle
**Everything is driven from Active ES Orders** - we migrate only the data needed to support these active services.

## Active Order Criteria
- **Status** IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
- **Project_Group__c** NOT LIKE '%PA MARKET DECOM%'

## Data Flow
```
Active Orders (current legacy BAN)
    │
    ├── Legacy Billing_Invoice__c.Id
    │       │
    │       └── New BBF BAN (Billing_Invoice__c where Legacy_ES_Id__c = legacy BAN Id)
    │               │
    │               └── Account__c → Accounts to migrate
    │                       │
    │                       └── Contacts to migrate
    │
    ├── Address_A__c → Locations to migrate
    │       │
    │       └── Off_Net__c (Location_1__c or Location_2__c)
    │
    └── Address_Z__c → Locations to migrate
            │
            └── Off_Net__c (Location_1__c or Location_2__c)
```

## Output
Excel workbook with sheets:
1. **Summary** - Overall migration scope and counts
2. **Active_Orders** - All qualifying orders (with BBF BAN mapping)
3. **BAN_Mapping** - Legacy BAN → New BBF BAN mapping
4. **Accounts** - Unique accounts to migrate
5. **Contacts** - Contacts for those accounts
6. **Locations** - Address_A and Address_Z locations
7. **Off_Net** - Off-Net records for migration locations
8. **Data_Quality** - Issues requiring attention (orders with BBF BANs only)
9. **Orders_Missing_BBF_BAN** - Orders whose legacy BAN has no new BBF BAN

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [3]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # 'login' for production, 'test' for sandbox

# Active Order Status Filter
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# PA Market Decom Exclusion (Project_Group__c filter)
PA_DECOM_FILTER = "PA MARKET DECOM"

# Output Configuration
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_bbf_migration_analysis_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")
print(f"   Active Statuses: {ACTIVE_STATUSES}")
print(f"   Excluding: Project_Group__c LIKE '%{PA_DECOM_FILTER}%'")
print(f"   Output: {OUTPUT_FILE}")

📋 Configuration loaded
   Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
   Excluding: Project_Group__c LIKE '%PA MARKET DECOM%'
   Output: es_bbf_migration_analysis_20260107_141523.xlsx


In [4]:
# === CONNECT TO ES SALESFORCE ===

print("=" * 80)
print("CONNECTING TO ES SALESFORCE")
print("=" * 80)

print("\n🔌 Connecting to ES...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

CONNECTING TO ES SALESFORCE

🔌 Connecting to ES...
✅ Connected to ES: everstream.my.salesforce.com


In [5]:
# === STEP 1: QUERY ALL ACTIVE ORDERS ===

print("\n" + "=" * 80)
print("STEP 1: QUERYING ACTIVE ORDERS")
print("=" * 80)

status_filter = "','".join(ACTIVE_STATUSES)

orders_query = f"""
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Address_A__c,
    Address_Z__c,
    Node__c,
    OpportunityId,
    Service_Start_Date__c,
    Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Service_ID__c,
    Vendor_Circuit_ID__c,
    Primary_Product_Family__c,
    Primary_Product_Name__c,
    Project_Group__c,
    CreatedDate,
    LastModifiedDate
FROM Order
WHERE Status IN ('{status_filter}')
ORDER BY Service_ID__c
"""

print("Querying all orders with active statuses...")
result = es_sf.query_all(orders_query)
all_orders_df = pd.DataFrame(result["records"])

# Flatten Account.Name
if "Account" in all_orders_df.columns:
    all_orders_df["Account_Name"] = all_orders_df["Account"].apply(
        lambda x: x.get("Name") if isinstance(x, dict) else None
    )
    all_orders_df = all_orders_df.drop(
        columns=["Account", "attributes"], errors="ignore"
    )
else:
    all_orders_df = all_orders_df.drop(columns=["attributes"], errors="ignore")

print(f"\n✅ Total orders with active statuses: {len(all_orders_df):,}")
print(f"\n=== Status Breakdown ===")
print(all_orders_df["Status"].value_counts().to_string())


STEP 1: QUERYING ACTIVE ORDERS
Querying all orders with active statuses...

✅ Total orders with active statuses: 17,970

=== Status Breakdown ===
Status
Activated                   17684
Disconnect in Progress        285
Suspended (Late Payment)        1


In [6]:
# === STEP 2: APPLY PA MARKET DECOM FILTER ===

print("\n" + "=" * 80)
print("STEP 2: FILTERING OUT PA MARKET DECOM")
print("=" * 80)

# Separate PA MARKET DECOM orders
pa_decom_mask = (
    all_orders_df["Project_Group__c"]
    .fillna("")
    .str.contains(PA_DECOM_FILTER, case=False)
)
pa_decom_orders_df = all_orders_df[pa_decom_mask].copy()
active_orders_df = all_orders_df[~pa_decom_mask].copy()

print(f"\n📊 Filter Results:")
print(f"   Total active status orders: {len(all_orders_df):,}")
print(f"   ❌ Excluded (PA MARKET DECOM): {len(pa_decom_orders_df):,}")
print(f"   ✅ In scope for migration: {len(active_orders_df):,}")

if len(pa_decom_orders_df) > 0:
    print(f"\n=== PA MARKET DECOM Project Groups ===")
    print(pa_decom_orders_df["Project_Group__c"].value_counts().head(10).to_string())


STEP 2: FILTERING OUT PA MARKET DECOM

📊 Filter Results:
   Total active status orders: 17,970
   ❌ Excluded (PA MARKET DECOM): 887
   ✅ In scope for migration: 17,083

=== PA MARKET DECOM Project Groups ===
Project_Group__c
PA MARKET DECOM                                    196
PHL PA MARKET DECOM                                 76
Uniti PA - Sprint Disco PA MARKET DECOM             58
PA MARKET DECOM - TMO BAN CHANGE - DO NOT TRACK     20
TMO RMSO Y3 $1250 RERATE  PA MARKET DECOM           18
Plymouth PA MARKET DECOM                            14
Pittston PA MARKET DECOM                            11
CPA- PIT PC PA MARKET DECOM                         11
PIT KEEP PA MARKET DECOM                            11
TMO RMSO Y3 $1250 RERATE PA MARKET DECOM             9


In [7]:
# === STEP 3: GET NEW BBF BANS (WITH LEGACY_ES_ID__c) ===

print("\n" + "=" * 80)
print("STEP 3: QUERYING NEW BBF BANS")
print("=" * 80)

bbf_ban_query = """
SELECT 
    Id,
    Name,
    Account__c,
    Account__r.Name,
    Legacy_ES_Id__c,
    BBF_Ban__c,
    Billing_Address_1__c,
    Billing_City__c,
    Billing_State__c,
    Billing_ZIP__c,
    Payment_Terms__c,
    Active_Billing__c
FROM Billing_Invoice__c
WHERE BBF_Ban__c = true
  AND Legacy_ES_Id__c != null
"""

print("Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...")
result = es_sf.query_all(bbf_ban_query)
bbf_bans_df = pd.DataFrame(result["records"])

if len(bbf_bans_df) > 0:
    # Flatten Account name
    if "Account__r" in bbf_bans_df.columns:
        bbf_bans_df["Account_Name"] = bbf_bans_df["Account__r"].apply(
            lambda x: x.get("Name") if isinstance(x, dict) else None
        )
        bbf_bans_df = bbf_bans_df.drop(
            columns=["Account__r", "attributes"], errors="ignore"
        )
    else:
        bbf_bans_df = bbf_bans_df.drop(columns=["attributes"], errors="ignore")

print(f"\n✅ Found {len(bbf_bans_df):,} new BBF BANs")

# Create mapping: Legacy BAN Id -> New BBF BAN record
legacy_to_bbf_ban = {}
for _, row in bbf_bans_df.iterrows():
    legacy_id = row["Legacy_ES_Id__c"]
    if legacy_id:
        legacy_to_bbf_ban[legacy_id] = row.to_dict()

print(f"   Created mapping for {len(legacy_to_bbf_ban):,} legacy BANs → new BBF BANs")


STEP 3: QUERYING NEW BBF BANS
Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...

✅ Found 2,505 new BBF BANs
   Created mapping for 2,505 legacy BANs → new BBF BANs


In [8]:
# === STEP 4: SIMULATE ORDER REASSIGNMENT & IDENTIFY GAPS ===

print("\n" + "=" * 80)
print("STEP 4: SIMULATING ORDER REASSIGNMENT TO NEW BBF BANS")
print("=" * 80)

# Track which orders can be mapped and which cannot
orders_with_bbf_ban = []
orders_missing_bbf_ban = []
orders_missing_any_ban = []

for _, order in active_orders_df.iterrows():
    legacy_ban_id = order.get("Billing_Invoice__c")

    if not legacy_ban_id:
        # Order has no BAN at all
        orders_missing_any_ban.append(order.to_dict())
    elif legacy_ban_id in legacy_to_bbf_ban:
        # Order can be mapped to new BBF BAN
        order_dict = order.to_dict()
        bbf_ban = legacy_to_bbf_ban[legacy_ban_id]
        order_dict["New_BBF_BAN_Id"] = bbf_ban["Id"]
        order_dict["New_BBF_BAN_Name"] = bbf_ban["Name"]
        order_dict["New_BBF_BAN_Account__c"] = bbf_ban["Account__c"]
        orders_with_bbf_ban.append(order_dict)
    else:
        # Order has legacy BAN but no corresponding new BBF BAN
        orders_missing_bbf_ban.append(order.to_dict())

print(f"\n📊 Order Reassignment Simulation Results:")
print(f"   ✅ Orders with new BBF BAN mapping: {len(orders_with_bbf_ban):,}")
print(f"   ⚠️  Orders missing new BBF BAN: {len(orders_missing_bbf_ban):,}")
print(f"   ❌ Orders with no BAN at all: {len(orders_missing_any_ban):,}")

# Create DataFrames
orders_ready_df = (
    pd.DataFrame(orders_with_bbf_ban) if orders_with_bbf_ban else pd.DataFrame()
)
orders_no_bbf_ban_df = (
    pd.DataFrame(orders_missing_bbf_ban) if orders_missing_bbf_ban else pd.DataFrame()
)
orders_no_ban_df = (
    pd.DataFrame(orders_missing_any_ban) if orders_missing_any_ban else pd.DataFrame()
)


STEP 4: SIMULATING ORDER REASSIGNMENT TO NEW BBF BANS

📊 Order Reassignment Simulation Results:
   ✅ Orders with new BBF BAN mapping: 13,519
   ⚠️  Orders missing new BBF BAN: 25
   ❌ Orders with no BAN at all: 3,539


In [9]:
# === STEP 5: IDENTIFY UNIQUE ACCOUNTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 5: IDENTIFYING ACCOUNTS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    # Get unique Account IDs from the new BBF BANs (not from Orders directly)
    unique_account_ids = (
        orders_ready_df["New_BBF_BAN_Account__c"].dropna().unique().tolist()
    )
    print(f"\nFound {len(unique_account_ids):,} unique Accounts from new BBF BANs")

    # Query Account details
    if unique_account_ids:
        # Chunk the query to avoid SOQL limits
        chunk_size = 150
        all_accounts = []

        for i in range(0, len(unique_account_ids), chunk_size):
            chunk = unique_account_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            account_query = f"""
            SELECT Id, Name, Type, Industry, BillingStreet, BillingCity, 
                   BillingState, BillingPostalCode, BillingCountry,
                   Phone, Website, BBF_New_Id__c
            FROM Account
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(account_query)
            all_accounts.extend(result["records"])

        accounts_df = pd.DataFrame(all_accounts)
        if "attributes" in accounts_df.columns:
            accounts_df = accounts_df.drop(columns=["attributes"])

        print(f"✅ Retrieved {len(accounts_df):,} Account records")

        # Check how many already migrated
        already_migrated = accounts_df["BBF_New_Id__c"].notna().sum()
        print(f"   Already migrated (BBF_New_Id__c populated): {already_migrated:,}")
        print(f"   Need to migrate: {len(accounts_df) - already_migrated:,}")
else:
    accounts_df = pd.DataFrame()
    print("⚠️  No orders ready for migration - cannot identify accounts")


STEP 5: IDENTIFYING ACCOUNTS TO MIGRATE

Found 2,257 unique Accounts from new BBF BANs
✅ Retrieved 2,257 Account records
   Already migrated (BBF_New_Id__c populated): 0
   Need to migrate: 2,257


In [10]:
# === STEP 6: IDENTIFY CONTACTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 6: IDENTIFYING CONTACTS TO MIGRATE")
print("=" * 80)

if len(accounts_df) > 0:
    account_ids = accounts_df["Id"].tolist()

    # Chunk the query
    chunk_size = 150
    all_contacts = []

    for i in range(0, len(account_ids), chunk_size):
        chunk = account_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        contact_query = f"""
        SELECT Id, AccountId, FirstName, LastName, Email, Phone, Title,
               MailingStreet, MailingCity, MailingState, MailingPostalCode,
               BBF_New_Id__c
        FROM Contact
        WHERE AccountId IN ('{ids_str}')
        """
        result = es_sf.query_all(contact_query)
        all_contacts.extend(result["records"])

    contacts_df = pd.DataFrame(all_contacts) if all_contacts else pd.DataFrame()
    if len(contacts_df) > 0 and "attributes" in contacts_df.columns:
        contacts_df = contacts_df.drop(columns=["attributes"])

    print(f"✅ Found {len(contacts_df):,} Contacts for migration accounts")

    if len(contacts_df) > 0:
        already_migrated = contacts_df["BBF_New_Id__c"].notna().sum()
        print(f"   Already migrated: {already_migrated:,}")
        print(f"   Need to migrate: {len(contacts_df) - already_migrated:,}")
else:
    contacts_df = pd.DataFrame()
    print("⚠️  No accounts identified - cannot query contacts")


STEP 6: IDENTIFYING CONTACTS TO MIGRATE
✅ Found 15,769 Contacts for migration accounts
   Already migrated: 0
   Need to migrate: 15,769


In [11]:
# === STEP 7: IDENTIFY LOCATIONS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 7: IDENTIFYING LOCATIONS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    # Get unique Address IDs from orders (Address_A__c and Address_Z__c)
    address_a_ids = orders_ready_df["Address_A__c"].dropna().unique().tolist()
    address_z_ids = orders_ready_df["Address_Z__c"].dropna().unique().tolist()

    # Combine and deduplicate
    all_address_ids = list(set(address_a_ids + address_z_ids))

    print(f"\nUnique locations referenced by orders with BBF BANs:")
    print(f"   Address_A (A Location): {len(address_a_ids):,}")
    print(f"   Address_Z (Z Location): {len(address_z_ids):,}")
    print(f"   Combined unique: {len(all_address_ids):,}")

    if all_address_ids:
        # Chunk the query
        chunk_size = 150
        all_addresses = []

        for i in range(0, len(all_address_ids), chunk_size):
            chunk = all_address_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            address_query = f"""
            SELECT Id, Name, Address__c, City__c, State__c, County__c, Zip__c,
                   Complete_Address__c, CLLI__c, Building_Status__c,
                   On_Net__c, BBF_New_Id__c
            FROM Address__c
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(address_query)
            all_addresses.extend(result["records"])

        locations_df = pd.DataFrame(all_addresses) if all_addresses else pd.DataFrame()
        if len(locations_df) > 0 and "attributes" in locations_df.columns:
            locations_df = locations_df.drop(columns=["attributes"])

        print(f"\n✅ Retrieved {len(locations_df):,} Location records")

        if len(locations_df) > 0:
            already_migrated = locations_df["BBF_New_Id__c"].notna().sum()
            print(f"   Already migrated: {already_migrated:,}")
            print(f"   Need to migrate: {len(locations_df) - already_migrated:,}")
    else:
        locations_df = pd.DataFrame()
else:
    locations_df = pd.DataFrame()
    all_address_ids = []
    print("⚠️  No orders ready - cannot identify locations")


STEP 7: IDENTIFYING LOCATIONS TO MIGRATE

Unique locations referenced by orders with BBF BANs:
   Address_A (A Location): 909
   Address_Z (Z Location): 10,916
   Combined unique: 11,329

✅ Retrieved 11,329 Location records
   Already migrated: 0
   Need to migrate: 11,329


In [12]:
# === STEP 8: IDENTIFY OFF_NET RECORDS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 8: IDENTIFYING OFF_NET RECORDS TO MIGRATE")
print("=" * 80)

if len(all_address_ids) > 0:
    # Query Off_Net__c where Location_1__c or Location_2__c is in our location list
    chunk_size = 100  # Smaller chunks for OR query
    all_offnet = []

    for i in range(0, len(all_address_ids), chunk_size):
        chunk = all_address_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        offnet_query = f"""
        SELECT Id, Name, 
               Location_1__c, Location_1_Address__c,
               Location_2__c, Location_2_Address__c,
               Off_Net_Vendor__c, Vendor_Name__c,
               Vendor_circuit_Id__c, Internal_Circuit_Id__c,
               Cost_MRC__c, Cost_NRC__c, Invoice_MRC__c,
               LEC_Order_Status__c, Off_Net_Type__c,
               Bandwidth__c, Circuit_Type__c,
               Term__c, Term_Agreement_Start_Date__c, Term_Agreement_End_Date__c,
               Vendor_Bill_Start_Date__c, Vendor_Bill_Stop_Date__c,
               SOF1__c
        FROM Off_Net__c
        WHERE Location_1__c IN ('{ids_str}')
           OR Location_2__c IN ('{ids_str}')
        """
        result = es_sf.query_all(offnet_query)
        all_offnet.extend(result["records"])

    # Deduplicate (same Off_Net could match both Location_1 and Location_2)
    offnet_df = pd.DataFrame(all_offnet) if all_offnet else pd.DataFrame()
    if len(offnet_df) > 0:
        if "attributes" in offnet_df.columns:
            offnet_df = offnet_df.drop(columns=["attributes"])
        offnet_df = offnet_df.drop_duplicates(subset=["Id"])

    print(f"\n✅ Found {len(offnet_df):,} Off_Net__c records for migration locations")

    if len(offnet_df) > 0:
        print(f"\n=== Off_Net Status Breakdown ===")
        if "LEC_Order_Status__c" in offnet_df.columns:
            print(offnet_df["LEC_Order_Status__c"].value_counts().to_string())
else:
    offnet_df = pd.DataFrame()
    print("⚠️  No locations identified - cannot query Off_Net records")


STEP 8: IDENTIFYING OFF_NET RECORDS TO MIGRATE

✅ Found 2,303 Off_Net__c records for migration locations

=== Off_Net Status Breakdown ===
LEC_Order_Status__c
Active                1881
Disco                  228
Pending                 84
Invoice Evaluation      50
Pending Disco           43
Unknown                 13
Soft Disco               2
NOC Review               2


In [13]:
# === STEP 9: DATA QUALITY ANALYSIS (ORDERS WITH BBF BANS ONLY) ===

print("\n" + "=" * 80)
print("STEP 9: DATA QUALITY ANALYSIS (Orders with BBF BANs only)")
print("=" * 80)

data_quality_issues = []

# Only analyze orders that have BBF BAN mapping (orders_ready_df)
if len(orders_ready_df) > 0:
    total_ready = len(orders_ready_df)

    # Check for missing Address_A__c
    missing_addr_a = orders_ready_df["Address_A__c"].isna().sum()
    if missing_addr_a > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Address_A__c",
                "Count": missing_addr_a,
                "Percentage": f"{missing_addr_a/total_ready*100:.1f}%",
                "Severity": "HIGH",
                "Impact": "Cannot set A_Location__c on BBF Service__c",
            }
        )

    # Check for missing Address_Z__c
    missing_addr_z = orders_ready_df["Address_Z__c"].isna().sum()
    if missing_addr_z > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Address_Z__c",
                "Count": missing_addr_z,
                "Percentage": f"{missing_addr_z/total_ready*100:.1f}%",
                "Severity": "MEDIUM",
                "Impact": "Cannot set Z_Location__c on BBF Service__c",
            }
        )

    # Check for missing Node__c
    missing_node = orders_ready_df["Node__c"].isna().sum()
    if missing_node > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Node__c",
                "Count": missing_node,
                "Percentage": f"{missing_node/total_ready*100:.1f}%",
                "Severity": "LOW",
                "Impact": "Can fix post-migration - A_Node__c/Z_Node__c optional",
            }
        )

    # Check for missing Service_Start_Date__c
    missing_start = orders_ready_df["Service_Start_Date__c"].isna().sum()
    if missing_start > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Service_Start_Date__c",
                "Count": missing_start,
                "Percentage": f"{missing_start/total_ready*100:.1f}%",
                "Severity": "MEDIUM",
                "Impact": "Review - may need for Active_Date__c",
            }
        )

    # Check for missing SOF_MRC__c
    missing_mrc = orders_ready_df["SOF_MRC__c"].isna().sum()
    if missing_mrc > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing SOF_MRC__c",
                "Count": missing_mrc,
                "Percentage": f"{missing_mrc/total_ready*100:.1f}%",
                "Severity": "MEDIUM",
                "Impact": "MRC value not available for Service__c",
            }
        )

data_quality_df = pd.DataFrame(data_quality_issues)

print(f"\n📊 Data Quality Summary (for {len(orders_ready_df):,} orders with BBF BANs):")
if len(data_quality_df) > 0:
    print(data_quality_df.to_string(index=False))
else:
    print("   ✅ No data quality issues found!")


STEP 9: DATA QUALITY ANALYSIS (Orders with BBF BANs only)

📊 Data Quality Summary (for 13,519 orders with BBF BANs):
                               Issue  Count Percentage Severity                                                Impact
         Orders missing Address_A__c     13       0.1%     HIGH            Cannot set A_Location__c on BBF Service__c
              Orders missing Node__c  11459      84.8%      LOW Can fix post-migration - A_Node__c/Z_Node__c optional
Orders missing Service_Start_Date__c   5905      43.7%   MEDIUM                  Review - may need for Active_Date__c


In [14]:
# === STEP 10: CREATE BAN MAPPING TABLE ===

print("\n" + "=" * 80)
print("STEP 10: CREATING BAN MAPPING TABLE")
print("=" * 80)

# Get unique legacy BANs from orders ready to migrate
if len(orders_ready_df) > 0:
    ban_mapping_data = []

    # Group orders by legacy BAN to get counts
    legacy_ban_counts = orders_ready_df.groupby("Billing_Invoice__c").size().to_dict()

    for legacy_id, bbf_ban in legacy_to_bbf_ban.items():
        order_count = legacy_ban_counts.get(legacy_id, 0)
        if order_count > 0:  # Only include BANs that have orders
            ban_mapping_data.append(
                {
                    "Legacy_BAN_Id": legacy_id,
                    "New_BBF_BAN_Id": bbf_ban["Id"],
                    "New_BBF_BAN_Name": bbf_ban["Name"],
                    "Account__c": bbf_ban["Account__c"],
                    "Account_Name": bbf_ban.get("Account_Name"),
                    "Order_Count": order_count,
                }
            )

    ban_mapping_df = pd.DataFrame(ban_mapping_data)
    print(f"\n✅ Created mapping for {len(ban_mapping_df):,} BANs")
    print(f"   Total orders covered: {ban_mapping_df['Order_Count'].sum():,}")
else:
    ban_mapping_df = pd.DataFrame()
    print("⚠️  No BAN mapping created - no orders ready")


STEP 10: CREATING BAN MAPPING TABLE

✅ Created mapping for 2,504 BANs
   Total orders covered: 13,519


In [15]:
# === STEP 11: GENERATE SUMMARY ===

print("\n" + "=" * 80)
print("STEP 11: MIGRATION SUMMARY")
print("=" * 80)

summary_data = [
    {"Category": "ORDERS", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Total Active Status Orders",
        "Count": len(all_orders_df),
        "Notes": "All orders with qualifying status",
    },
    {
        "Category": "",
        "Metric": "Excluded (PA MARKET DECOM)",
        "Count": len(pa_decom_orders_df),
        "Notes": "Project_Group__c contains 'PA MARKET DECOM'",
    },
    {
        "Category": "",
        "Metric": "In Scope for Migration",
        "Count": len(active_orders_df),
        "Notes": "Active orders not in PA MARKET DECOM",
    },
    {
        "Category": "",
        "Metric": "Ready (have BBF BAN mapping)",
        "Count": len(orders_ready_df),
        "Notes": "Can be migrated now",
    },
    {
        "Category": "",
        "Metric": "Missing BBF BAN",
        "Count": len(orders_no_bbf_ban_df),
        "Notes": "Need new BBF BAN created",
    },
    {
        "Category": "",
        "Metric": "Missing ANY BAN",
        "Count": len(orders_no_ban_df),
        "Notes": "CRITICAL - cannot migrate",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "BANS", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "New BBF BANs Available",
        "Count": len(bbf_bans_df),
        "Notes": "Billing_Invoice__c with BBF_Ban__c=true",
    },
    {
        "Category": "",
        "Metric": "BANs with Orders to Migrate",
        "Count": len(ban_mapping_df) if len(ban_mapping_df) > 0 else 0,
        "Notes": "BANs that have active orders",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "ACCOUNTS", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Accounts to Migrate",
        "Count": len(accounts_df),
        "Notes": "Unique accounts from BBF BANs",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "CONTACTS", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Contacts to Migrate",
        "Count": len(contacts_df),
        "Notes": "Contacts for migration accounts",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "LOCATIONS", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Locations to Migrate",
        "Count": len(locations_df),
        "Notes": "Address_A + Address_Z from orders with BBF BANs",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "OFF_NET", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Off_Net Records to Migrate",
        "Count": len(offnet_df),
        "Notes": "Off_Net for migration locations",
    },
]

summary_df = pd.DataFrame(summary_data)

print("\n" + summary_df.to_string(index=False))


STEP 11: MIGRATION SUMMARY

 Category                       Metric Count                                           Notes
   ORDERS                                                                                   
            Total Active Status Orders 17970               All orders with qualifying status
            Excluded (PA MARKET DECOM)   887     Project_Group__c contains 'PA MARKET DECOM'
                In Scope for Migration 17083            Active orders not in PA MARKET DECOM
          Ready (have BBF BAN mapping) 13519                             Can be migrated now
                       Missing BBF BAN    25                        Need new BBF BAN created
                       Missing ANY BAN  3539                       CRITICAL - cannot migrate
                                                                                            
     BANS                                                                                   
                New BBF BANs Available  2

In [16]:
# === STEP 12: EXPORT TO EXCEL ===

print("\n" + "=" * 80)
print("STEP 12: EXPORTING TO EXCEL")
print("=" * 80)

wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# --- SHEET 1: Summary ---
ws_summary.append(["ES → BBF Migration Analysis"])
ws_summary["A1"].font = Font(bold=True, size=16)
ws_summary.append([f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"])
ws_summary.append([])

headers = ["Category", "Metric", "Count", "Notes"]
ws_summary.append(headers)
for col, header in enumerate(headers, 1):
    cell = ws_summary.cell(row=4, column=col)
    cell.font = header_font
    cell.fill = header_fill

for _, row in summary_df.iterrows():
    ws_summary.append([row["Category"], row["Metric"], row["Count"], row["Notes"]])

ws_summary.column_dimensions["A"].width = 15
ws_summary.column_dimensions["B"].width = 35
ws_summary.column_dimensions["C"].width = 12
ws_summary.column_dimensions["D"].width = 50

print("   ✅ Summary sheet created")

# --- SHEET 2: Active Orders ---
if len(orders_ready_df) > 0:
    ws_orders = wb.create_sheet("Active_Orders")
    order_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "New_BBF_BAN_Id",
        "New_BBF_BAN_Name",
        "Address_A__c",
        "Address_Z__c",
        "SOF_MRC__c",
        "Service_Start_Date__c",
    ]
    order_cols = [c for c in order_cols if c in orders_ready_df.columns]

    ws_orders.append(order_cols)
    for col, header in enumerate(order_cols, 1):
        cell = ws_orders.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in orders_ready_df[order_cols].iterrows():
        ws_orders.append(list(row))

    ws_orders.freeze_panes = "A2"
    print(f"   ✅ Active_Orders sheet created ({len(orders_ready_df):,} rows)")

# --- SHEET 3: BAN Mapping ---
if len(ban_mapping_df) > 0:
    ws_ban = wb.create_sheet("BAN_Mapping")
    headers = list(ban_mapping_df.columns)
    ws_ban.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_ban.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in ban_mapping_df.iterrows():
        ws_ban.append(list(row))

    ws_ban.freeze_panes = "A2"
    print(f"   ✅ BAN_Mapping sheet created ({len(ban_mapping_df):,} rows)")

# --- SHEET 4: Accounts ---
if len(accounts_df) > 0:
    ws_acct = wb.create_sheet("Accounts")
    headers = list(accounts_df.columns)
    ws_acct.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_acct.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in accounts_df.iterrows():
        ws_acct.append(list(row))

    ws_acct.freeze_panes = "A2"
    print(f"   ✅ Accounts sheet created ({len(accounts_df):,} rows)")

# --- SHEET 5: Contacts ---
if len(contacts_df) > 0:
    ws_contact = wb.create_sheet("Contacts")
    headers = list(contacts_df.columns)
    ws_contact.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_contact.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in contacts_df.iterrows():
        ws_contact.append(list(row))

    ws_contact.freeze_panes = "A2"
    print(f"   ✅ Contacts sheet created ({len(contacts_df):,} rows)")

# --- SHEET 6: Locations ---
if len(locations_df) > 0:
    ws_loc = wb.create_sheet("Locations")
    headers = list(locations_df.columns)
    ws_loc.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_loc.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in locations_df.iterrows():
        ws_loc.append(list(row))

    ws_loc.freeze_panes = "A2"
    print(f"   ✅ Locations sheet created ({len(locations_df):,} rows)")

# --- SHEET 7: Off_Net ---
if len(offnet_df) > 0:
    ws_offnet = wb.create_sheet("Off_Net")
    headers = list(offnet_df.columns)
    ws_offnet.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_offnet.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill

    for _, row in offnet_df.iterrows():
        ws_offnet.append(list(row))

    ws_offnet.freeze_panes = "A2"
    print(f"   ✅ Off_Net sheet created ({len(offnet_df):,} rows)")

# --- SHEET 8: Data Quality ---
if len(data_quality_df) > 0:
    ws_dq = wb.create_sheet("Data_Quality")
    headers = list(data_quality_df.columns)
    ws_dq.append(headers)
    for col, header in enumerate(headers, 1):
        cell = ws_dq.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = PatternFill(
            start_color="FF4444", end_color="FF4444", fill_type="solid"
        )

    for _, row in data_quality_df.iterrows():
        ws_dq.append(list(row))

    severity_colors = {
        "CRITICAL": "FF6666",
        "HIGH": "FFAA66",
        "MEDIUM": "FFFF66",
        "LOW": "66FF66",
    }
    for row_idx in range(2, len(data_quality_df) + 2):
        severity = ws_dq.cell(row=row_idx, column=4).value
        if severity in severity_colors:
            for col in range(1, len(headers) + 1):
                ws_dq.cell(row=row_idx, column=col).fill = PatternFill(
                    start_color=severity_colors[severity],
                    end_color=severity_colors[severity],
                    fill_type="solid",
                )

    ws_dq.freeze_panes = "A2"
    print(f"   ✅ Data_Quality sheet created ({len(data_quality_df):,} issues)")

# --- SHEET 9: Orders Missing BBF BAN ---
if len(orders_no_bbf_ban_df) > 0:
    ws_missing = wb.create_sheet("Orders_Missing_BBF_BAN")
    cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "SOF_MRC__c",
    ]
    cols = [c for c in cols if c in orders_no_bbf_ban_df.columns]

    ws_missing.append(cols)
    for col, header in enumerate(cols, 1):
        cell = ws_missing.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = PatternFill(
            start_color="FF4444", end_color="FF4444", fill_type="solid"
        )

    for _, row in orders_no_bbf_ban_df[cols].iterrows():
        ws_missing.append(list(row))

    ws_missing.freeze_panes = "A2"
    print(
        f"   ✅ Orders_Missing_BBF_BAN sheet created ({len(orders_no_bbf_ban_df):,} rows)"
    )

# --- SHEET 10: PA MARKET DECOM (Excluded) ---
if len(pa_decom_orders_df) > 0:
    ws_excluded = wb.create_sheet("Excluded_PA_DECOM")
    cols = ["Id", "Name", "Service_ID__c", "Status", "Project_Group__c", "Account_Name"]
    cols = [c for c in cols if c in pa_decom_orders_df.columns]

    ws_excluded.append(cols)
    for col, header in enumerate(cols, 1):
        cell = ws_excluded.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = PatternFill(
            start_color="808080", end_color="808080", fill_type="solid"
        )

    for _, row in pa_decom_orders_df[cols].iterrows():
        ws_excluded.append(list(row))

    ws_excluded.freeze_panes = "A2"
    print(f"   ✅ Excluded_PA_DECOM sheet created ({len(pa_decom_orders_df):,} rows)")

wb.save(OUTPUT_FILE)
print(f"\n✅ Excel file saved: {OUTPUT_FILE}")


STEP 12: EXPORTING TO EXCEL
   ✅ Summary sheet created
   ✅ Active_Orders sheet created (13,519 rows)
   ✅ BAN_Mapping sheet created (2,504 rows)
   ✅ Accounts sheet created (2,257 rows)
   ✅ Contacts sheet created (15,769 rows)
   ✅ Locations sheet created (11,329 rows)
   ✅ Off_Net sheet created (2,303 rows)
   ✅ Data_Quality sheet created (3 issues)
   ✅ Orders_Missing_BBF_BAN sheet created (25 rows)
   ✅ Excluded_PA_DECOM sheet created (887 rows)

✅ Excel file saved: es_bbf_migration_analysis_20260107_141523.xlsx


In [17]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION DATA ANALYSIS COMPLETE")
print("=" * 80)

print(
    f"""
📊 SUMMARY
{'─'*40}
Active Orders (in scope):     {len(active_orders_df):,}
  ├─ Ready to migrate:        {len(orders_ready_df):,}
  ├─ Missing BBF BAN:         {len(orders_no_bbf_ban_df):,}
  └─ Missing ANY BAN:         {len(orders_no_ban_df):,}

Records to Migrate:
  ├─ Accounts:                {len(accounts_df):,}
  ├─ Contacts:                {len(contacts_df):,}
  ├─ Locations:               {len(locations_df):,}
  ├─ Off_Net:                 {len(offnet_df):,}
  └─ BANs (with orders):      {len(ban_mapping_df):,}

Excluded:
  └─ PA MARKET DECOM:         {len(pa_decom_orders_df):,}

Output: {OUTPUT_FILE}
"""
)

if len(orders_no_bbf_ban_df) > 0 or len(orders_no_ban_df) > 0:
    print("⚠️  ACTION REQUIRED:")
    if len(orders_no_bbf_ban_df) > 0:
        print(f"   • {len(orders_no_bbf_ban_df):,} orders need new BBF BANs created")
    if len(orders_no_ban_df) > 0:
        print(f"   • {len(orders_no_ban_df):,} orders have no BAN - CANNOT migrate")
    print("\n   Review the Excel output for details.")
else:
    print("✅ All orders ready for migration!")


MIGRATION DATA ANALYSIS COMPLETE

📊 SUMMARY
────────────────────────────────────────
Active Orders (in scope):     17,083
  ├─ Ready to migrate:        13,519
  ├─ Missing BBF BAN:         25
  └─ Missing ANY BAN:         3,539

Records to Migrate:
  ├─ Accounts:                2,257
  ├─ Contacts:                15,769
  ├─ Locations:               11,329
  ├─ Off_Net:                 2,303
  └─ BANs (with orders):      2,504

Excluded:
  └─ PA MARKET DECOM:         887

Output: es_bbf_migration_analysis_20260107_141523.xlsx

⚠️  ACTION REQUIRED:
   • 25 orders need new BBF BANs created
   • 3,539 orders have no BAN - CANNOT migrate

   Review the Excel output for details.


---
## Next Steps

### If Orders Missing BBF BAN > 0:
1. Review the `Orders_Missing_BBF_BAN` sheet
2. Identify the legacy BANs that need new BBF BANs created
3. Create the new BBF BANs in ES with `BBF_Ban__c = true` and `Legacy_ES_Id__c` = legacy BAN Id
4. Re-run this analysis

### Migration Execution Order:
1. **Account** - `es_bbf_account_migration.ipynb`
2. **Contact** - `es_bbf_contact_migration.ipynb`
3. **Location** - `es_bbf_location_migration.ipynb`
4. **BAN** - `es_bbf_ban_migration.ipynb`
5. **Service** - `es_bbf_service_migration.ipynb` (Order → Service__c)
6. **Service_Charge** - `es_bbf_service_charge_migration.ipynb` (OrderItem → Service_Charge__c)
7. **Off_Net** - `es_bbf_offnet_migration.ipynb`
8. **BAN_Contact** - (auto-created or derived)

### Manual Setup (outside notebooks):
- Product2
- Pricebook2  
- PricebookEntry